<a href="https://colab.research.google.com/github/elisaboccini/AMDproject/blob/main/V3NOSPLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Market Basket Analysis**

Kaggle API

In [1]:
from google.colab import files
files.upload() #upload kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"elisaboccini","key":"3024605213579b61d22c8ab4775d25f4"}'}

In [2]:
#kaggle API 
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -q ashirwadsangwan/imdb-dataset
!unzip imdb-dataset.zip

kaggle.json
Archive:  imdb-dataset.zip
  inflating: name.basics.tsv.gz      
  inflating: name.basics.tsv/name.basics.tsv  
  inflating: title.akas.tsv.gz       
  inflating: title.akas.tsv/title.akas.tsv  
  inflating: title.basics.tsv.gz     
  inflating: title.basics.tsv/title.basics.tsv  
  inflating: title.principals.tsv.gz  
  inflating: title.principals.tsv/title.principals.tsv  
  inflating: title.ratings.tsv.gz    
  inflating: title.ratings.tsv/title.ratings.tsv  


Spark Session

In [3]:
#Spark session creation

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz 
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 212.3MB 60kB/s 
     |████████████████████████████████| 204kB 48.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=5b0a500cc4de383a196abbf1598dfdbf9f449fb8c2ee02ab2b06595f41522742
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [5]:
import findspark
findspark.init('spark-3.1.1-bin-hadoop2.7')

In [6]:
from pyspark.sql import SparkSession

In [7]:
#spark = SparkSession.builder.master("local[*]").getOrCreate()
#try with 4 g to avoid garbage collection
spark = SparkSession.builder.master("local")\
        .appName("MBA")\
        .config('spark.ui.port', '4050')\
        .config("spark.driver.memory", "15g") \
        .config("spark.driver.maxResultSize", "1g") \
        .config("spark.sql.broadcastTimeout", "10h") \
        .getOrCreate()

In [8]:
import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [9]:
#sc = spark.sparkContext

Dataset

In [10]:
principals = spark.read.csv('title.principals.tsv.gz', sep="\t", header=True, inferSchema=True)
#akas = spark.read.csv('title.akas.tsv.gz', sep="\t", header=True, inferSchema=True)
titlebasics = spark.read.csv('title.basics.tsv.gz', sep="\t", header=True, inferSchema=True)
#ratings = spark.read.csv('title.ratings.tsv.gz', sep="\t", header=True, inferSchema=True)
namesbasics = spark.read.csv('name.basics.tsv.gz', sep="\t", header=True, inferSchema=True)

In [11]:
principals.createOrReplaceTempView("principals")
#akas.createOrReplaceTempView("akas")
titlebasics.createOrReplaceTempView("titlebasics")
#ratings.createOrReplaceTempView("ratings")
namesbasics.createOrReplaceTempView("namesbasics")

In [12]:
# drop duplicates
titlebasics.dropDuplicates((['tconst'])).count() # 6326545
namesbasics.dropDuplicates((['nconst'])).count() #9711022

9711022

In [13]:
#which are the types of items collected in the titlebasics db?
spark.sql('''SELECT distinct (titleType)
              FROM titlebasics''').show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
+------------+



In [14]:
#Taking only actors (not crew) and only movies and shorts (not tvSeries, tvMiniSeries, tvEpisode, tvSpecial,video, videogames)
data = spark.sql('''SELECT tb.tconst, tb.primaryTitle, nb.primaryName, nb.nconst 
                          FROM titlebasics as tb LEFT JOIN principals as p ON tb.tconst = p.tconst INNER JOIN namesbasics as nb ON p.nconst = nb.nconst
                          WHERE ((p.category = 'actor' OR p.category = 'actress') AND (tb.titleType = 'movie' OR tb.titleType = 'tvMovie' OR tb.titleType = 'short' OR tb.titleType = 'tvShort')) ''')

data.createOrReplaceTempView('data')
data.show(5)
data.count() #3686467-> this also reduces a lot the size of the db (from 14 mln to 3 mln)

+---------+-----------------+--------------+---------+
|   tconst|     primaryTitle|   primaryName|   nconst|
+---------+-----------------+--------------+---------+
|tt0083109| The Cabbage Soup|Louis de Funès|nm0000086|
|tt0062120|The Little Bather|Louis de Funès|nm0000086|
|tt0057422|    Squeak-squeak|Louis de Funès|nm0000086|
|tt0043691|     The Gamblers|Louis de Funès|nm0000086|
|tt0335439| Station mondaine|Louis de Funès|nm0000086|
+---------+-----------------+--------------+---------+
only showing top 5 rows



3686467

In [15]:
spark.sql('''select count(distinct tconst) 
            from data''').show() # 974512 -> we have actually 1/3 of the unique movies we had before!

+----------------------+
|count(DISTINCT tconst)|
+----------------------+
|                974512|
+----------------------+



In [16]:
#lets see if we can group by primarytitle (if the distinct titles are equivalent to n of tconst)
spark.sql('''select count(distinct primaryTitle) 
            from data''').show() #756111 -> there are homonyms in movies and shorts

+----------------------------+
|count(DISTINCT primaryTitle)|
+----------------------------+
|                      756111|
+----------------------------+



FP-Growth: pre-processing

In [17]:
#baskets creation
from pyspark.sql.functions import collect_set

moviesasbaskets = data.groupBy('tconst','primaryTitle').agg(collect_set('primaryName').alias('cast')) #metti un alias
moviesasbaskets.createOrReplaceTempView('moviesasbaskets')

moviesasbaskets.show(n=5) #974512 -> correct! we have a basket for each unique tconst

+---------+--------------------+--------------------+
|   tconst|        primaryTitle|                cast|
+---------+--------------------+--------------------+
|tt0000007|Corbett and Court...|[James J. Corbett...|
|tt0000378|          Les vagues|[Julie Souplet, G...|
|tt0000621|   That Fatal Sneeze|[Gertie Potter, T...|
|tt0000653|A Calamitous Elop...|[John R. Cumpson,...|
|tt0000841|       The Day After|[Marion Leonard, ...|
+---------+--------------------+--------------------+
only showing top 5 rows



In [18]:
#concatenate using hyphen
# to have info about the title but still maintaining the basket form (so just one col for the basket and the corresponding
# list of itemsets)
from pyspark.sql import functions as f

moviesasbaskets = moviesasbaskets.withColumn('movie', f.concat(f.col('tconst'),f.lit('-'),f.col('primaryTitle')))

moviesasbaskets = moviesasbaskets.select('movie', 'cast')
moviesasbaskets.show(5)

+--------------------+--------------------+
|               movie|                cast|
+--------------------+--------------------+
|tt0000007-Corbett...|[James J. Corbett...|
|tt0000378-Les vagues|[Julie Souplet, G...|
|tt0000621-That Fa...|[Gertie Potter, T...|
|tt0000653-A Calam...|[John R. Cumpson,...|
|tt0000841-The Day...|[Marion Leonard, ...|
+--------------------+--------------------+
only showing top 5 rows



In [19]:
# Caching aka persisting-> otherwise randomsplit will lead to inconsistencies, since it changes the randomsplit each
# time you perform an operation on the splitted data
#moviesasbaskets = moviesasbaskets.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
# apply the randomSplit() 
#MBAtrain,MBAtest = moviesasbaskets.randomSplit([0.9, 0.1], seed=13)

FP-Growth: Model implementation

In [52]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="cast", minSupport=0.0001, minConfidence=0)
modelFP = fpGrowth.fit(moviesasbaskets)  
#excluding the episodes we consider frequent an actor if it appears in (distinct tconst = 974512)*0.0001 97 films 
#reasonable

FP-Growth: Results

In [53]:
# Display frequent itemsets.-> NB this result includes all size items!
from pyspark.sql.functions import col, asc,desc
freqItemsets = modelFP.freqItemsets
freqItemsets.createOrReplaceTempView("freqItemsets")
freqItemsets.orderBy(col('freq').desc()).show()
#the most frequent actor is no more nm10120013 appearing in > 5000 films BUT Mel Blanc with 925 movies
#however taking as minSupport 0.0005 only 5 actors are retrieved and therefore no association rules
#therefore try again with a minSupport of 0.0001-> now it works
#try also to set minSUpport as 0.0003 as in the Apriori analysis

+--------------------+----+
|               items|freq|
+--------------------+----+
|         [Mel Blanc]|1030|
|  [Matsunosuke Onoe]| 932|
|      [Brahmanandam]| 797|
|       [Adoor Bhasi]| 585|
|      [Eddie Garcia]| 510|
|       [Jack Mercer]| 453|
|        [Prem Nazir]| 436|
|   [Jack Richardson]| 434|
|         [Lee Moran]| 429|
|      [Sung-il Shin]| 412|
|       [Eddie Lyons]| 406|
|      [Paquito Diaz]| 393|
|      [Oliver Hardy]| 392|
|  [Masayoshi Nogami]| 390|
|[J. Warren Kerrigan]| 383|
|         [Mammootty]| 381|
|    [Aachi Manorama]| 356|
|           [Bahadur]| 348|
|          [Mohanlal]| 346|
|[Gilbert M. 'Bron...| 342|
+--------------------+----+
only showing top 20 rows



In [54]:
# Display generated association rules.
associationRules = modelFP.associationRules
associationRules.createOrReplaceTempView("associationRules")
associationRules.show(n=5) 

+--------------------+-------------------+-------------------+------------------+--------------------+
|          antecedent|         consequent|         confidence|              lift|             support|
+--------------------+-------------------+-------------------+------------------+--------------------+
|[Curly Howard, La...|       [Moe Howard]|                1.0|4490.8387096774195|1.067200814356313...|
|      [Larry Storch]|  [Norma MacMillan]|            0.78125|        6797.65625|1.026154629188763...|
|[Kijaku Ôtani, Ma...| [Kitsuraku Arashi]|  0.743421052631579| 5406.512961508249|1.159554730983302...|
|[Kijaku Ôtani, Ma...|[Suminojo Ichikawa]| 0.6776315789473685|  5368.78134360291|1.056939268064426E-4|
| [Arthur V. Johnson]|[Florence Lawrence]|0.38022813688212925|1414.2629088903723|1.026154629188763...|
+--------------------+-------------------+-------------------+------------------+--------------------+
only showing top 5 rows



In [55]:
#now only two rows in association rules having confidence equal to 1
spark.sql("""select (*)
           from associationRules
           where confidence = '1.0' 
           limit 20""").show()
#We obtain that where there are Curly Howard and Larry Fine there is Moe Howard and where there are Curly Howard and Moe Howard there is Larry Fine

+--------------------+------------+----------+------------------+--------------------+
|          antecedent|  consequent|confidence|              lift|             support|
+--------------------+------------+----------+------------------+--------------------+
|[Curly Howard, La...|[Moe Howard]|       1.0|4490.8387096774195|1.067200814356313...|
|[Curly Howard, Mo...|[Larry Fine]|       1.0| 4575.173708920188|1.067200814356313...|
+--------------------+------------+----------+------------------+--------------------+



In [56]:
#for a better understanding ###INTERESTING
spark.sql("""select antecedent as `antecedent (if)`, consequent as `consequent (then)`, confidence
              from associationRules
              order by confidence desc
              limit 20""").show()

+--------------------+--------------------+------------------+
|     antecedent (if)|   consequent (then)|        confidence|
+--------------------+--------------------+------------------+
|[Curly Howard, La...|        [Moe Howard]|               1.0|
|[Curly Howard, Mo...|        [Larry Fine]|               1.0|
|[Pauline Bush, Ja...|[J. Warren Kerrigan]|0.9919354838709677|
|        [Larry Fine]|        [Moe Howard]|0.9906103286384976|
|      [Curly Howard]|        [Moe Howard]|0.9904761904761905|
|      [Curly Howard]|        [Larry Fine]|0.9904761904761905|
|[Bebe Daniels, 'S...|      [Harold Lloyd]|0.9863945578231292|
|[Harold Lloyd, 'S...|      [Bebe Daniels]|0.9863945578231292|
|[Kitsuraku Arashi...|  [Matsunosuke Onoe]|0.9826086956521739|
|[Suminojo Ichikaw...|  [Matsunosuke Onoe]|0.9809523809523809|
|[Lillian Peacock,...|      [Billy Franey]|0.9805825242718447|
|[Harold Lloyd, Be...|    ['Snub' Pollard]|0.9797297297297297|
|  [Kitsuraku Arashi]|  [Matsunosuke Onoe]|0.9776119402

In [57]:
# transform examines the input items against all the association rules and summarize the consequents as prediction
# it would be useful to divide the db in training and test set in order to actually do predictions
association = modelFP.transform(moviesasbaskets)
association.show()

+--------------------+--------------------+--------------------+
|               movie|                cast|          prediction|
+--------------------+--------------------+--------------------+
|tt0000007-Corbett...|[James J. Corbett...|                  []|
|tt0000378-Les vagues|[Julie Souplet, G...|                  []|
|tt0000621-That Fa...|[Gertie Potter, T...|                  []|
|tt0000653-A Calam...|[John R. Cumpson,...|                  []|
|tt0000841-The Day...|[Marion Leonard, ...| [Florence Lawrence]|
|tt0001628-The Fou...|    [Norma Talmadge]|                  []|
|tt0001693-The Imm...|[Francis Ford, Wi...|                  []|
|tt0001820-The Out...|[Gilbert M. 'Bron...|[Harry Todd, Fred...|
|tt0002453-The Rec...|[Harold Lockwood,...|                  []|
|tt0002870-Flood Tide|[Marie Eline, Lil...|                  []|
|tt0003172-The Mou...|[Gladden James, E...|                  []|
|tt0003404-The Sta...|[Theodor Burghard...|                  []|
|tt0003601-The Adv...|[Jo

In [58]:
#Doing predictions for MBAtest2
association.createOrReplaceTempView('association')
predFP= spark.sql("""select *
                      from association
                      where size(prediction) >= 1 """)
predFP.createOrReplaceTempView('predFP')
predFP.show() #718 rows in pred_ex2

+--------------------+--------------------+--------------------+
|               movie|                cast|          prediction|
+--------------------+--------------------+--------------------+
|tt0000841-The Day...|[Marion Leonard, ...| [Florence Lawrence]|
|tt0001820-The Out...|[Gilbert M. 'Bron...|[Harry Todd, Fred...|
|tt0003649-Back to...|[Oliver Hardy, Ma...|       [Stan Laurel]|
|tt0014064-Frozen ...|[Katherine Grant,...|      [Oliver Hardy]|
|tt0027899-Little ...|[Mae Questel, Jac...|      [Jackson Beck]|
|tt0029282-My Arti...|[Mae Questel, Jac...|      [Jackson Beck]|
|tt0142813-Run 'Em...|[Marie Mosquini, ...|[Sammy Brooks, Be...|
|tt0148079-Doing W...|[Gwen Davies, Jac...|[Mae Questel, Jac...|
|   tt0150681-Galaxia|[Jack Mercer, Day...|[Mae Questel, Jac...|
|tt0151497-The Mar...|[Jack Mercer, Mar...|[Mae Questel, Jac...|
|tt0152146-Ration ...|       [Jack Mercer]|[Mae Questel, Jac...|
|tt0156953-Riders ...|       [Fred Church]|[Gilbert M. 'Bron...|
|tt0186869-The Bet...|   

In [59]:
#focus the analysis for frequent itemset with one actor only (frequent actors)
#NOW filter freqItemsets by those having items = 1 -> most frequent actors banally
mostfrequentactor= spark.sql("""select items, freq
                                    from freqItemsets
                                    where size(items) = 1
                                    order by freq desc""")
mostfrequentactor.show(n=5)
mostfrequentactor.count()

+------------------+----+
|             items|freq|
+------------------+----+
|       [Mel Blanc]|1030|
|[Matsunosuke Onoe]| 932|
|    [Brahmanandam]| 797|
|     [Adoor Bhasi]| 585|
|    [Eddie Garcia]| 510|
+------------------+----+
only showing top 5 rows



1342

In [60]:
#now analysis for frequent itemset with two actors
mostfrequent2actors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) = 2
                                      order by freq desc""")
mostfrequent2actors.show(n=5)
mostfrequent2actors.count()

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Eddie Lyons, Lee...| 318|
|[Prem Nazir, Adoo...| 236|
|[Mae Questel, Jac...| 216|
|[Larry Fine, Moe ...| 211|
|[J. Warren Kerrig...| 186|
+--------------------+----+
only showing top 5 rows



56

In [61]:
#now analysis for frequent itemset with three actors
mostfrequent3actors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) = 3
                                      order by freq desc""")
mostfrequent3actors.show(n=5)
mostfrequent3actors.count() 

+--------------------+----+
|               items|freq|
+--------------------+----+
|[Harold Lloyd, Be...| 145|
|[Pauline Bush, J....| 123|
|[Milburn Morante,...| 119|
|[Kitsuraku Arashi...| 113|
|[Winifred Greenwo...| 112|
+--------------------+----+
only showing top 5 rows



9

In [63]:
#for more than 3 actors there is only an itemset, containing 4 actors, considered frequent according to our parameter and having freq = 88
#now analysis for frequent itemset with three actors
mostfrequentnactors= spark.sql("""select items, freq
                                      from freqItemsets
                                      where size(items) > 3
                                      order by freq desc""")
mostfrequentnactors.show(n=5)
mostfrequentnactors.count()

+-----+----+
|items|freq|
+-----+----+
+-----+----+



0

In [64]:
mostfrequentnactors.toPandas()['items'].tolist() 
#just to visualize that the 4 most frequent actors are [['Sammy Brooks', 'Harold Lloyd', 'Bebe Daniels', "'Snub' Pollard"]]
#appearing together in 88 films

[]